#### Generación de Temas a través de GenAI según bigramas y trigramas

In [72]:
import warnings
warnings.filterwarnings('ignore')

In [73]:
archivo_csv_importar = 'new_ds/cluster_8_sub_0_top_words.csv'
archivo_csv_exportar = 'new_ds/cluster_8_sub_0_genai.csv'
titulo_columna_analizar = 'grams'
id_columna = 'cluster'
titulo_columna_nueva = 'topic'
gramas = 0 # 2, 3, 0 (ambos)
grabar_archivo = True


In [74]:
# importamos librerías
import numpy as np
import pandas as pd
import json
import requests
import re
import time
import random

### Importar Datasets

In [75]:
#importo datasets
df = pd.read_csv(archivo_csv_importar, encoding='utf8')

In [76]:
df.head()

,cluster,bigrams,trigrams
0,0,"[(('home', 'banking'), 16), (('tarjeta', 'cred...","[(('gustaria', 'cobrar', 'jubilacion'), 7), ((..."
1,1,"[(('menu', 'deuda'), 2), (('volver', 'menu'), ...","[(('volver', 'menu', 'deudas'), 2), (('deudas'..."
2,2,"[(('saldo', 'tarjeta'), 8), (('libre', 'deuda'...","[(('refinanciar', 'saldo', 'tarjeta'), 2), (('..."
3,3,"[(('llego', 'plata'), 3), (('debitaron', 'plat...","[(('plata', 'caja', 'dieron'), 1), (('retire',..."
4,4,"[(('constancia', 'cbu'), 15), (('descargar', '...","[(('descargar', 'constancia', 'cbu'), 13), (('..."


### GENAI 

In [77]:
## GENAI ##
model = "mistralai/mixtral-8x7b-instruct-v01"
api_key = "" #os.getenv("API_KEY", None)
ibm_cloud_url = "" #os.getenv("IBM_CLOUD_URL", None)
project_id = "" #os.getenv("PROJECT_ID", None)
url_cloud = ""
creds = {
    "url": ibm_cloud_url,
    "apikey": api_key 
}
token = ''
# propiedades
max_tokens_limit = 8192
max_tokens = 4096
temperature = 0.7
top_p = 1.0
random_seed = random.randrange(1,99)
total_tokens_used = 0
response_seconds = 0.0
total_response_seconds = 0.0
avg_response_seconds = 0.0

    

In [78]:
def generateTokenIBM(myApik):    
    #curl -X POST 'https://iam.cloud.ibm.com/identity/token' 
    url = "https://iam.cloud.ibm.com/identity/token"
    # -d 'grant_type=urn:ibm:params:oauth:grant-type:apikey & apikey=${myApik}
    data = {
        "grant_type": "urn:ibm:params:oauth:grant-type:apikey",
        "apikey": myApik
    }
    # -H 'Content-Type: application/x-www-form-urlencoded'
    headers = {"Content-Type": "application/x-www-form-urlencoded"}
    #curl -X POST 'https://iam.cloud.ibm.com/identity/token' -H 'Content-Type: application/x-www-form-urlencoded' 
    # -d 'grant_type=urn:ibm:params:oauth:grant-type:apikey&apikey=${myApik}'
    response = requests.post(url, data=data, headers=headers)  ### La request de CURL.
    access_token = None
    # Verifica si la solicitud fue exitosa (código de estado 200)
    if response.status_code == 200:
        # Parsea la respuesta JSON
        #print("response: ",response,"\n\n")
        json_data = json.loads(response.text)
        # Extrae el valor de 'access_token' de la respuesta JSON
        access_token = json_data.get('access_token')
    else:
        print(f"La solicitud falló con el código de estado {response.status_code}.")
        print(f"Response.text: ")
        json_data = json.loads(response.text)
        print(json_data)
    return access_token

In [114]:
def clean_text(text):
    """
    text: a string
    return: modified initial string
    """
    text = str(text)
    # lowercase text
    text = text.lower() 
    # replace REPLACE_BY_SPACE_RE symbols by space in text
    text = REPLACE_BY_SPACE_RE.sub(' ', text) 
    # Remove the XXXX values
    text = text.replace('x', '') 
    # Remove white space
    text = REMOVE_NUM.sub('', text)
    #  delete symbols which are in BAD_SYMBOLS_RE from text
    #text = BAD_SYMBOLS_RE.sub('', text) 
    # delete stopwords from text
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) 
    # removes any words composed of less than 2 or more than 21 letters
    text = ' '.join(word for word in text.split() if (len(word) >= 2 and len(word) <= 21))
    # Stemming the words
    #text = ' '.join([stemmer.stem(word) for word in text.split()])
    # Stemming the words
    if forma_tokenizar == 's':
        text = ' '.join([stemmer.stem(str(word)) for word in text.split()])
    elif forma_tokenizar == 'l': #lemmatization
        text = ' '.join(x.lemma_ for x in nlp(text))
    return text

# Limpia texto de caracteres extendidos y espacios en blanco
def clean_text_message(texto):
    texto_str = str(texto)
    # Definir un patrón que permita caracteres ASCII imprimibles y caracteres extendidos del español
    caracteres_permitidos = re.compile(r'[^\x20-\x7E\u00A1-\u00FF]')
    texto_limpio = caracteres_permitidos.sub('', texto_str)
    # Eliminar espacios en blanco innecesarios al principio y al final del texto
    texto_limpio = texto_limpio.strip()
    # Reemplazar más de dos espacios en blanco consecutivos por un solo espacio
    texto_limpio = re.sub(r'\s{2,}', ' ', texto_limpio)
    # Reemplaza basura de json
    return texto_limpio

# extrae la pregunta
def extraer_entre_caracteres(cadena, caracter_inicio, caracter_fin):
    try:
        inicio = cadena.index(caracter_inicio)
        fin = cadena.index(caracter_fin, inicio) + 1
        return cadena[inicio:fin]
    except ValueError:
        return cadena

# extrae texto json con [] o con {}
def extract_text_between_characters(input_string, characters=""):
    # Utiliza una expresión regular para encontrar todo el texto entre "{...}"
    if characters=="{}":
        pattern = re.compile(r'\{(.*?)\}')
        matches = pattern.findall(input_string)
    elif characters=="[]":
        pattern = re.compile(r'\[(.*?)\]')
        matches = pattern.findall(input_string)
    elif characters=="¿?":
        pattern = re.compile(r'\¿(.*?)\?')
        matches = pattern.findall(input_string)
    else:
        matches = input_string
    return matches

# convierte una lista a json string
def list_to_json(json_lst):
    json_str = json.dumps(json_lst, ensure_ascii=False) if type(json_lst) == list else json_lst
    json_str = clean_text(json_str)
    return json_str

# convierte un json string a lista
def json_to_list(json_str, characters=""):
    #json_str = clean_text_message(json_str)
    decoded_json_str = json_str.encode().decode('utf-8-sig')
    #decoded_json_str = clean_text(decoded_json_str)
    decoded_json_str = decoded_json_str.replace("```json", "")
    decoded_json_str = decoded_json_str.replace("```", "")
    decoded_json_str = decoded_json_str.replace("}[{", "[{")
    decoded_json_str = decoded_json_str.replace("['{", "[{")
    decoded_json_str = decoded_json_str.replace("}']", "}]")
    #if characters=="":
    #    decoded_json_str = extract_text_between_characters(decoded_json_str, characters)
    #print(decoded_json_str)
    try:
        json_lst = json.loads(decoded_json_str)
    except:
        json_lst = decoded_json_str
    return json_lst


In [80]:
def chat_with_model(prompt_txt, model, temperature, max_tokens=250):

    start_time = time.time()
    
    if model == "mistralai/mixtral-8x7b-instruct-v01" or model == "mistralai/mistral-large":
        prompt = prompt_txt #mixtral_array2string(messages) # convierte array de prompts a string mixtral
        url = url_cloud
        headers = {
        	"Accept": "application/json",
        	"Content-Type": "application/json",
        	"Authorization": "Bearer " + token
        }
        data = {
            "model_id": model,
            "project_id": project_id,
            #"messages": messages, 
            # en el input debe pasarse un string con todo el prompt, ej: <s>[INST] bla bla [INST]</s>
            # sino tira este error: input must be of type string
            #"input": f"""<s>[INST] {prompt_system} [INST]</s> 
            #[INST] {tmp_prompt} [INST]</s> """, 
            "input": prompt,
            "parameters": {
                "decoding_method": "sample", "temperature": temperature, "top_p": top_p, "top_k": 50, "typical_p": 1
                #"decoding_method": "greedy"
                ,"min_new_tokens": 1
                ,"max_new_tokens": max_tokens
                ,"random_seed": random_seed
                ,"stop_sequences": ["?"]
                ,"repetition_penalty": 1.1
            },
            "moderations": {}
        }
        # ejecuta la api
        response = requests.post(url, headers=headers, json=data)
        if response.status_code != 200:
        	raise Exception("Non-200 response: " + str(response.text))
        response_data = response.json()
        generated_text = response_data['results'][0]['generated_text']
        total_tokens = str(int(response_data['results'][0]['input_token_count']) + int(response_data['results'][0]['generated_token_count']))
    
    else: # gpt
        generated_text = "error de modelo llm elegido"
        total_tokens = 0

    end_time = time.time()
    # mide tiempo de respuesta
    response_time = end_time - start_time
    
    return generated_text, int(total_tokens), round(response_time, 2)


In [81]:
# Función para interactuar con la API de ChatGPT
def ejecutar_modelo(string_lista, gramas=3): #gramas 2, 3, 0 (ambos)
    
    # Crear el prompt para la API
    prompt = f"""<s>[INST] Dada la siguiente lista con palabras en español y cantidad de repeticiones de esas palabras en el siguiente formato: 
    [(palabras, repeticiones), (palabras, repeticiones), (palabras, repeticiones)] del rubro banca, 
    solo responde de qué se trata el tema siguiendo estas instrucciones: 
    - Responde en menos de 10 palabras únicamente y en idioma español latinoamericano. 
    - No agregues comentarios ni caracteres extendidos.
    - No repitas las palabras: bancarios, bancarias, bancas, tema, temática.
    Lista de palabras y repeticiones: "{string_lista}". Temática: [/INST] """

    prompt_bigramas = f"""<s>[INST] Dada la siguiente lista de bigramas con palabras en español y cantidad de repeticiones de esas palabras en el siguiente formato: 
    [((palabra 1, palabra 2), repeticiones), ((palabra 1, palabra 2), repeticiones), ...] del rubro bancario, 
    solo responde de qué se trata el tema siguiendo estas instrucciones: 
    - Responde en menos de 12 palabras únicamente tomando los 10 primeros bigramas con más repeticiones como más importantes.
    - Responde en idioma español latinoamericano. 
    - No agregues comentarios ni caracteres extendidos.
    - No repitas las palabras: bancarios, bancarias, bancas, tema, temática.
    Lista de bigramas de palabras y repeticiones: "{string_lista}". Temática: [/INST] """
    
    prompt_trigramas = f"""<s>[INST] Dada la siguiente lista de trigramas con palabras en español y cantidad de repeticiones de esas palabras en el siguiente formato: 
    [((palabra 1, palabra 2, palabra 3), repeticiones), ((palabra 1, palabra 2, palabra 3), repeticiones), ...] del rubro bancario, 
    solo responde de qué se trata el tema siguiendo estas instrucciones: 
    - Responde en menos de 12 palabras únicamente tomando los 10 primeros trigramas con más repeticiones como más importantes.
    - Responde en idioma español latinoamericano. 
    - No agregues comentarios ni caracteres extendidos.
    - No repitas las palabras: bancarios, bancarias, bancas, tema, temática.
    Lista de trigramas de palabras y repeticiones: "{string_lista}". Temática: [/INST] """

    prompt_gramas = f"""<s>[INST] Dada una lista de bigramas y trigramas con palabras en español y cantidad de repeticiones de esas palabras del rubro bancario en el siguiente formato: 
    Bigramas: [((palabra 1, palabra 2), repeticiones), ((palabra 1, palabra 2), repeticiones), ...] 
    Trigramas: [((palabra 1, palabra 2, palabra 3), repeticiones), ((palabra 1, palabra 2, palabra 3), repeticiones), ...] , 
    solo responde de qué se trata el tema siguiendo estas instrucciones: 
    - Responde en menos de 12 palabras únicamente tomando los 10 primeros bigramas y 10 primeros trigramas con más repeticiones como las palabras más importantes.
    - Responde en idioma español latinoamericano. 
    - No agregues comentarios ni caracteres extendidos.
    - No repitas las palabras: bancarios, bancarias, bancas, tema, temática.
    La siguiente es la lista de bigramas y trigramas de palabras y repeticiones: "{string_lista}". Temática: [/INST] """
    
    if gramas==2:
        response, total_tokens_used, response_seconds = chat_with_model(prompt_bigramas, model, temperature, max_tokens)
    elif gramas==3:
        response, total_tokens_used, response_seconds = chat_with_model(prompt_trigramas, model, temperature, max_tokens)
    elif gramas==0:
        response, total_tokens_used, response_seconds = chat_with_model(prompt_gramas, model, temperature, max_tokens)
    #question = clean_text(response)
    #response_clean = extraer_entre_caracteres(response, "{", "}")
    return response, total_tokens_used, response_seconds

    

In [82]:
# Función principal para iterar sobre las conversaciones y generar el DataFrame
def proceso_genai_df(df):

    genai = []
    # Iterar sobre el DataFrame
    for index, row in df.iterrows():
        if gramas == 0:
            str_gramas = f"Bigramas: {str(row['bigrams'])} y Trigramas: {str(row['trigrams'])} "
            # Enviar la conversación a la API y obtener la respuesta
            response, total_tokens_used, response_seconds = ejecutar_modelo(str_gramas, gramas)
        else: # 2, 3
            # Enviar la conversación a la API y obtener la respuesta
            response, total_tokens_used, response_seconds = ejecutar_modelo(str(row[titulo_columna_analizar]), gramas)
        # reemplazar comas por nada
        response =  response.replace(',', '') 
        if gramas == 0:
            genai.append({"cluster": row["cluster"], "bigrams": row["bigrams"], "trigrams": row["trigrams"], titulo_columna_nueva: response, "tokens": total_tokens_used, "seconds": response_seconds})
        else: # 2, 3
            genai.append({"cluster": row["cluster"], "grams": row["grams"], titulo_columna_nueva: response, "tokens": total_tokens_used, "seconds": response_seconds})
    # agrega columna nueva al dataframe con los valores generados
    df_genai = pd.DataFrame(genai)

    return df_genai

In [83]:
## GENERACION DE RESPUESTAS
#genera token
token = generateTokenIBM(api_key)
# Analizar los textos usando LLM y obtener el DataFrame con los resultados
df_resultado = proceso_genai_df(df)

In [84]:
df_resultado

,cluster,bigrams,trigrams,topic,tokens,seconds
0,0,"[(('home', 'banking'), 16), (('tarjeta', 'cred...","[(('gustaria', 'cobrar', 'jubilacion'), 7), ((...",Óperaciones bancarias en línea y cobro de jubi...,968,1.42
1,1,"[(('menu', 'deuda'), 2), (('volver', 'menu'), ...","[(('volver', 'menu', 'deudas'), 2), (('deudas'...",Órdenes financieros y problemas con tarjetas d...,963,1.45
2,2,"[(('saldo', 'tarjeta'), 8), (('libre', 'deuda'...","[(('refinanciar', 'saldo', 'tarjeta'), 2), (('...",Trata sobre gestión de deudas y tarjetas banca...,981,1.54
3,3,"[(('llego', 'plata'), 3), (('debitaron', 'plat...","[(('plata', 'caja', 'dieron'), 1), (('retire',...",Órdenes bancarias: plata débitos cajeros impue...,969,1.35
4,4,"[(('constancia', 'cbu'), 15), (('descargar', '...","[(('descargar', 'constancia', 'cbu'), 13), (('...",Órdenes bancarias para descargar constancias y...,1070,2.76
5,5,"[(('blanqueo', 'claves'), 1), (('bloqueo', 'cl...","[(('llamen', 'bloqueo', 'claves'), 1), (('menu...",Seguridad y recuperación claves en banca digi...,787,1.95
6,6,"[(('pago', 'modo'), 2), (('modo', 'aplicacion'...","[(('aplicacion', 'figura', 'pago'), 1), (('rea...",Óperaciones bancarias pagos devoluciones onlin...,949,1.48
7,7,"[(('tarjeta', 'master'), 8), (('pedir', 'tarje...","[(('tarjeta', 'suspendida', 'mora'), 2), (('ta...",Órdenes y problemas con tarjetas de crédito ba...,987,1.22
8,8,"[(('desconocimiento', 'consumo'), 3), (('desco...","[(('hecho', 'desconocimiento', 'compra'), 1), ...",Desconocimiento bancario en operaciones y rec...,739,1.05
9,9,"[(('impuesto', 'pais'), 10), (('estan', 'cobra...","[(('google', 'free', 'fire'), 1), (('rg', 'pes...",Óptima para análisis: impuestos retenciones pa...,953,1.28


In [85]:
df_resultado.to_csv(archivo_csv_exportar, sep=',', index=False, encoding='utf-8')